# Analysis of the risk vs returns of bitcoin

## Setup

In [ ]:
import matplotlib.colorbar as mpl_cbar
import matplotlib.colors as mpl_cols
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import seaborn as sns

In [ ]:
# Get bitcoin price df and set date as index
df_btc = pd.read_csv("../data/BTC.csv", index_col="date", parse_dates=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get OHLC average as price
df_btc["price"] = df_btc[["open", "high", "low", "close"]].mean(axis=1)
df_btc.drop(columns=["open", "high", "low", "close"], inplace=True)

## Volatility vs returns (90-day rolling values)

In [ ]:
# Get daily price change and 90-day moving average
df_btc["price_change"] = df_btc["price"].pct_change()
df_btc["price_change_90d_ma"] = df_btc["price_change"].rolling(window=90).mean()

In [ ]:
# Get 90-day rolling volatily with the standard deviation method (for consistency)
df_btc["volatility_90d"] = df_btc["price_change"].rolling(window=90).std()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Normalize years to map colors
norm = mpl_cols.Normalize(vmin=df_btc.index.year.min(), vmax=df_btc.index.year.max())
cmap = plt.cm.viridis_r

sns.scatterplot(data=df_btc, x="volatility_90d", y="price_change_90d_ma", hue=df_btc.index.year, palette=cmap, alpha=0.7, linewidth=0.2, legend=False)
plt.axhline(y=0, color="red", linewidth=1, linestyle="--")

plt.title("Volatility vs returns of bitcoin")
plt.xlabel("Volatility (90-day window)")
plt.ylabel("Average daily returns (90-day window)")

# Create the colorbar
cax, _ = mpl_cbar.make_axes(ax, location="right", pad=0.01)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = fig.colorbar(sm, cax=cax)

save_chart_as_png("3_BTC_vlt_vs_returns")

In [ ]:
# Pearson correlation coefficient betweent volatility and returns
df_btc["volatility_90d"].corr(df_btc["price_change_90d_ma"]).round(2)

**Key takeaways:**
- ...